In [1]:
import requests
from bs4 import BeautifulSoup
import pyexcel as pe

def parse_match_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    players_data = []

    players_data.append(['Номер', 'Имя', 'Мин', 'Старт', 'Замена', 'ГП', 'Гол', 'ЖК', 'КК'])

    protocols = soup.find_all('ul', class_='protocol__list')

    if len(protocols) < 4:
        print("Не удалось найти необходимые блоки с данными о матче.")
        return

    home_starting = protocols[0]
    home_substitutes = protocols[2]

    away_starting = protocols[1]
    away_substitutes = protocols[3]

    def process_players(protocol, is_starting):
        for player in protocol.find_all('li', class_='protocol__item'):
            player_number = player.find('span', class_='protocol__number-text').text.strip() if player.find('span', class_='protocol__number-text') else 'N/A'
            player_name = player.find('div', class_='protocol__name').text.strip() if player.find('div', class_='protocol__name') else 'N/A'
            played_minutes = 90 if is_starting else 0
            
            goals = 0
            for event in player.find_all('li', class_='protocol__event', title='Гол'):
                event_number = event.find_next_sibling('li', class_='protocol__event-number')
                if event_number:
                    goals += int(event_number.text.strip().replace('x', ''))
                else:
                    goals += 1
            
            assists = 0
            for event in player.find_all('li', class_='protocol__event', title='Ассистент'):
                event_number = event.find_next_sibling('li', class_='protocol__event-number')
                if event_number:
                    assists += int(event_number.text.strip().replace('x', ''))
                else:
                    assists += 1
            
            yellow_card = 1 if player.find('li', class_='protocol__event', title='Желтая карточка') else 0
            red_card = 1 if player.find('li', class_='protocol__event', title='Красная карточка') else 0
            sub_in = 0
            sub_out = 0

            events = player.find('ul', class_='protocol__events')
            if events:
                for event in events.find_all('li', class_='protocol__event'):
                    if 'protocol__event--out' in event['class']:
                        played_minutes = int(event.text.strip().replace("'", ""))
                        sub_out = 1
                    elif 'protocol__event--in' in event['class']:
                        played_minutes = 90 - int(event.text.strip().replace("'", ""))
                        sub_in = 1

            if sub_out == 1:
                sub_in = 0

            players_data.append([player_number, player_name, played_minutes, 1 if is_starting else 0, sub_in, assists, goals, yellow_card, red_card])

    players_data.append(['Хозяева', '', '', '', '', '', '', '', ''])
    process_players(home_starting, True)
    process_players(home_substitutes, False)

    players_data.append([''] * 9)
    players_data.append([''] * 9)
    players_data.append([''] * 9)

    players_data.append(['Гости', '', '', '', '', '', '', '', ''])
    process_players(away_starting, True)
    process_players(away_substitutes, False)

    sheet = pe.Sheet(players_data)
    
    sheet.save_as('/Users/zaytsev/Desktop/match_protocol.xlsx')

parse_match_data('https://yflrussia.ru/match/3808937')